In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
import psycopg2 as pg2

In [5]:
conn = pg2.connect(dbname='nhl', user='postgres', host='localhost', port='5435')
cur = conn.cursor()

In [12]:
query = '''
SELECT * 
FROM pst;
'''

query_c = '''
SELECT *
FROM contracts;
'''

In [7]:
cur.execute(query)

In [10]:
for row in cur:
    print(row)

(4124, 103, 'Roberto Luongo', 'Goaltender', 12, 5333333, 63999996, 9.39, datetime.date(2009, 9, 2), 2009, 30, None, 'UFA', 'UFA')
(2837, 155, 'Ryan Miller', 'Goaltender', 3, 6000000, 18000000, 8.7, datetime.date(2014, 7, 1), 2014, 33, 'Vancouver Canucks', 'UFA', 'UFA')
(2832, 155, 'Ryan Miller', 'Goaltender', 2, 2000000, 4000000, 2.67, datetime.date(2017, 7, 1), 2017, 36, 'Anaheim Ducks', 'UFA', 'UFA')
(4078, 156, 'Craig Anderson', 'Goaltender', 3, 4200000, 12600000, 6.09, datetime.date(2014, 8, 25), 2014, 33, None, 'UFA', 'UFA')
(4079, 156, 'Craig Anderson', 'Goaltender', 2, 4750000, 9500000, 6.33, datetime.date(2017, 9, 29), 2017, 36, None, 'UFA', 'UFA')


In [8]:
pst = list(cur)

In [15]:
len(pst)

9673

In [13]:
cur.execute(query_c)
contracts = list(cur)

In [14]:
pst_cols = ['Season_Player', 'Player', 'Season', 'Position', 'GP', 'TOI', 'Goals', 
                'Total Assists', 'First Assists', 'Second Assists', 'Total Points',
               'IPP', 'Shots', 'SH%', 'iCF', 'iFF', 'iSCF', 'iHDCF', 'Rush Attempts',
               'Rebounds Created', 'PIM', 'Total Penalties', 'Minor', 'Major',
               'Misconduct', 'Penalties Drawn', 'Giveaways', 'Takeaways', 'Hits',
               'Hits Taken', 'Shots Blocked', 'Faceoffs Won', 'Faceoffs Lost',
               'Faceoffs %'
                ]

In [15]:
df = pd.DataFrame(pst, columns = pst_cols)

In [16]:
df.head()

,Season_Player,Player,Season,Position,GP,TOI,Goals,Total Assists,First Assists,Second Assists,...,Misconduct,Penalties Drawn,Giveaways,Takeaways,Hits,Hits Taken,Shots Blocked,Faceoffs Won,Faceoffs Lost,Faceoffs %
0,2017 Connor McDavid,Connor McDavid,2017,C,82,1766.80,41,67,39,28,...,0,36,67,111,28,118,46,376,533,41.36
1,2017 Claude Giroux,Claude Giroux,2017,C,82,1669.80,34,68,33,35,...,0,26,44,34,30,54,23,638,450,58.64
2,2017 Nikita Kucherov,Nikita Kucherov,2017,R,80,1585.72,39,61,33,28,...,0,36,79,66,31,92,15,3,2,60.00
3,2017 Evgeni Malkin,Evgeni Malkin,2017,C,78,1481.22,42,56,32,24,...,1,30,73,75,48,98,32,460,592,43.73
4,2017 Nathan MacKinnon,Nathan MacKinnon,2017,C,74,1472.98,39,58,38,20,...,1,24,41,36,38,74,22,485,672,41.92


In [17]:
df.iloc[0]

Season_Player       2017 Connor McDavid
Player                   Connor McDavid
Season                             2017
Position                              C
GP                                   82
TOI                              1766.8
Goals                                41
Total Assists                        67
First Assists                        39
Second Assists                       28
Total Points                        108
IPP                               88.52
Shots                               275
SH%                               14.91
iCF                                 434
iFF                                 357
iSCF                                329
iHDCF                               173
Rush Attempts                        18
Rebounds Created                     34
PIM                                  14
Total Penalties                       7
Minor                                 7
Major                                 0
Misconduct                            0


In [21]:
(df.Goals / df.Shots)[0]

0.14909090909090908

In [22]:
df['SH%'][0]

14.91

In [43]:
#can't sum SH%, faceoff%, or IPP so we're ignoring them for now to make things simpler

In [18]:
dfsummable = df.drop(['SH%', 'Faceoffs %', 'IPP'], axis=1)

In [30]:
df['SH%'].mean()

7.388850408353132

In [40]:
df[df['Player'] == 'Alex Ovechkin'][['SH%', 'Season']]

,SH%,Season
10,13.80,2017
907,10.54,2016
1789,12.56,2015
2676,13.42,2014
3563,13.21,2013
4443,14.55,2012
5316,12.54,2011
6179,8.72,2010
7064,13.59,2009
7941,10.61,2008


In [19]:
dfsummable_wind = dfsummable.copy()
dfsummable_car = dfsummable.copy()

In [20]:
window = 3

In [21]:
summable_stats = ['GP', 'TOI', 'Goals', 
                'Total Assists', 'First Assists', 'Second Assists', 'Total Points',
               'Shots', 'iCF', 'iFF', 'iSCF', 'iHDCF', 'Rush Attempts',
               'Rebounds Created', 'PIM', 'Total Penalties', 'Minor', 'Major',
               'Misconduct', 'Penalties Drawn', 'Giveaways', 'Takeaways', 'Hits',
               'Hits Taken', 'Shots Blocked', 'Faceoffs Won', 'Faceoffs Lost']

In [57]:
dfsummable.sample(10)

,Season_Player,Player,Season,Position,GP,TOI,Goals,Total Assists,First Assists,Second Assists,...,Major,Misconduct,Penalties Drawn,Giveaways,Takeaways,Hits,Hits Taken,Shots Blocked,Faceoffs Won,Faceoffs Lost
1842,2015 Tomas Plekanec,Tomas Plekanec,2015,C,82,1519.6300,14,40,21,19,...,0,0,19,45,15,60,66,46,804,811
2147,2015 Cody Franson,Cody Franson,2015,D,59,993.4000,4,13,6,7,...,0,0,2,31,14,61,39,56,0,0
392,2017 Leo Komarov,Leo Komarov,2017,C,74,1168.2300,7,12,4,8,...,1,0,9,42,47,219,66,48,112,100
2520,2015 Michal Jordan,Michal Jordan,2015,D,36,545.9000,1,0,0,0,...,0,0,3,8,6,24,59,34,0,0
6096,2011 Pierre-Luc Letourneau-Leblond,Pierre-Luc Letourneau-Leblond,2011,R,3,14.5667,0,0,0,0,...,0,1,1,1,0,2,0,0,0,0
7659,2009 Jay Harrison,Jay Harrison,2009,D,38,559.2670,1,5,1,4,...,6,0,9,11,6,67,35,46,0,0
6563,2010 Jeff Tambellini,Jeff Tambellini,2010,L,62,730.5670,9,8,5,3,...,0,0,5,10,11,113,73,21,19,16
7329,2009 Sami Salo,Sami Salo,2009,D,68,1406.1700,9,19,7,12,...,0,0,9,29,16,43,50,60,0,0
6029,2011 Cody Bass,Cody Bass,2011,C,14,127.2170,0,1,1,0,...,6,0,10,4,4,22,14,7,5,3
6199,2010 Lubomir Visnovsky,Lubomir Visnovsky,2010,D,81,1967.7200,18,50,18,32,...,0,0,8,45,30,52,76,116,0,0


In [22]:
contracts = pd.read_csv('../data/Contract_Details_PuckPedia_Mar_2019_Confidential.csv')

In [23]:
def clean_data(file = None):
    #load in dataframe from csv
    if file:
        df = pd.read_csv(file)
    else:
        df = pd.read_csv('../data/Contract_Details_PuckPedia_Mar_2019_Confidential.csv')

    #Drop columns I won't be using
    df.drop(['buyout_id', 'base_salary', 'p_bonuses', 's_bonuses',
            'total_salary'], axis=1, inplace=True)

    #rename Total Value column to a format I prefer
    df['total_value'] = df[' Total Value ']
    df.drop(' Total Value ', axis=1, inplace=True)

    #create a column for full player name
    df['Player'] = df['first_name'] + ' ' + df['last_name']

    #make a function to convert money strings to ints and apply
    money_to_int = lambda x: int(x.strip().strip('$()').replace(',', ''))
    df['cap_hit'] = df['cap_hit'].apply(money_to_int)
    df['total_value'] = df['total_value'].apply(money_to_int)

    #convert signing date and birthdate to date format and pull out the signing year
    df.signing_date = pd.to_datetime(df.signing_date)
    df.birthdate = pd.to_datetime(df.birthdate)
    df['signing_year'] = pd.DatetimeIndex(df.signing_date).year

    #get age at contract signing_date (not accounting for leap days)
    df['signing_age'] = df.signing_date - df.birthdate
    df.signing_age = df.signing_age.apply(lambda x: x.days // 365)

    #convert signing_date to a date instead of a datetime
    df['signing_date'] = pd.DatetimeIndex(df['signing_date']).date

    #remove row that had NaN for signing year (Defenseman Fyodor Tyutin had
    #the contract listed in other rows also so no data is lost
    df.drop(6991, axis=0, inplace=True)

    #Make signing year an int instead of a float
    df.signing_year = df.signing_year.apply(int)

    #Manually entered league salary cap history
    scap = {
            2005: 39000000,
            2006: 44000000,
            2007: 50300000,
            2008: 56700000,
            2009: 56800000,
            2010: 59400000,
            2011: 64300000,
            2012: 60000000,
            2013: 64300000,
            2014: 69000000,
            2015: 71400000,
            2016: 73000000,
            2017: 75000000,
            2018: 79500000,
            2019: 83000000,
           }

    #Add a column for the total salary cap in a contract's year signed
    df['signing_year_cap'] = df['signing_year'].apply(lambda x: scap[x])
    # and a column for the percentage of the cap in the year signed
    df['cap_pct'] = round(100 * df.cap_hit / df.signing_year_cap, 2)

    #convert birthdate to pandas datetime
    df.birthdate = pd.to_datetime(df.birthdate)

    #convert current season and contract_end to single year ints for ease of
    #calculations
    df.season = df.season.apply(lambda x: int(x[:4]))
    df.contract_end = df.contract_end.apply(lambda x: int(x[:4]))

    #Drop some non-standard situations (i.e. suspensions, season-opening IR,
    # salary retention adjustments)
    df.drop(df[df.first_name.map(len) > 15].index, inplace=True)

    #Drop one extremely young player
    df.drop(df[df.ufa_year > 2030].index, inplace = True)

    #Drop where ufa_year is null
    df.drop(df[df.ufa_year.isnull()].index, inplace=True)
    #Turn the remaining ufa years into integers
    df.ufa_year = df.ufa_year.apply(int)

    #Drop any contracts less than league minimum in 2009
    df.drop(df[df.cap_hit < 500000].index, inplace=True)

    #Replace the position names for two players whose positions don't match the rest
    df.replace(['RW', 'LW'], ['Right Wing', 'Left Wing'], inplace=True)

    #drop entry level contracts
    df.drop(df[df.contract_level == 'entry_level'].index, inplace=True)

    #Add a column for whether the contract was signed as UFA or RFA
    df['signing_status'] = df['signing_year'] >= df['ufa_year']
    df['signing_status'] = df['signing_status'].apply(ufa_check)


    return df

#auxillary function to check if contract was signed as UFA or RFA
def ufa_check(x):
    if x == True:
        return 'UFA'
    else:
        return 'RFA'


In [24]:
contracts = clean_data()

In [25]:
contracts = contracts.groupby('contract_id').head(1)

In [26]:
contracts.Player.unique()

array(['Roberto Luongo', 'Ryan Miller', 'Craig Anderson', ...,
       'Parker Milner', 'Dominik Uher', 'Linden Vey'], dtype=object)

In [27]:
len(contracts.Player.unique())

1053

In [28]:
contracts = contracts[contracts.position != 'Goaltender']

In [29]:
len(contracts.Player.unique())

935

In [31]:
from collections import defaultdict

In [32]:
t = defaultdict(pd.DataFrame)

In [97]:
allstats = defaultdict(pd.DataFrame)
allsumstats = defaultdict(pd.DataFrame)
allmeanstats = defaultdict(pd.DataFrame)

for player in contracts['Player'].unique():
    allstats[player] = dfsummable[df.Player == player]
    allstats[player].sort_values(by='Season', inplace=True)
    allstats[player]['Season_index'] = pd.date_range(end='2018', 
                                               periods = allstats[player].shape[0], 
                                               freq='Y')
    allstats[player].set_index(allstats[player].Season_index, inplace=True)
    allsumstats[player] = (allstats[player][summable_stats].rolling(window=3, min_periods=1)
                  .agg(np.sum))
    allmeanstats[player] = (allstats[player][summable_stats].rolling(window=3, min_periods=1)
              .agg(np.mean))
    allmeanstats[player]['SH%'] = allsumstats[player]['Shots'] / allsumstats[player]['Goals']
    allmeanstats[player]['Faceoffs %'] = allsumstats[player]['Faceoffs Won'] / (
        allsumstats[player]['Faceoffs Won'] + allsumstats[player]['Faceoffs Lost'])

/Users/colinshanahan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/colinshanahan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [98]:
len(allstats)

935

In [99]:
allstats['Tomas Hertl']

,Season_Player,Player,Season,Position,GP,TOI,Goals,Total Assists,First Assists,Second Assists,...,Misconduct,Penalties Drawn,Giveaways,Takeaways,Hits,Hits Taken,Shots Blocked,Faceoffs Won,Faceoffs Lost,Season_index
Season_index,,,,,,,,,,,,,,,,,,,,,
2013-12-31,2013 Tomas Hertl,Tomas Hertl,2013,C,37,567.150,15,10,5,5,...,0,10,18,18,39,54,13,29,22,2013-12-31
2014-12-31,2014 Tomas Hertl,Tomas Hertl,2014,C,82,1193.300,13,18,6,12,...,0,15,64,29,106,98,33,38,45,2014-12-31
2015-12-31,2015 Tomas Hertl,Tomas Hertl,2015,C,81,1293.820,21,25,12,13,...,0,24,41,35,90,109,60,256,201,2015-12-31
2016-12-31,2016 Tomas Hertl,Tomas Hertl,2016,C,49,843.967,10,12,7,5,...,0,6,18,30,76,60,42,245,229,2016-12-31
2017-12-31,2017 Tomas Hertl,Tomas Hertl,2017,C,79,1430.020,22,24,14,10,...,1,19,45,60,84,118,71,587,569,2017-12-31


In [100]:
allsumstats['Tomas Hertl']

,GP,TOI,Goals,Total Assists,First Assists,Second Assists,Total Points,Shots,iCF,iFF,...,Major,Misconduct,Penalties Drawn,Giveaways,Takeaways,Hits,Hits Taken,Shots Blocked,Faceoffs Won,Faceoffs Lost
Season_index,,,,,,,,,,,,,,,,,,,,,
2013-12-31,37.0,567.150,15.0,10.0,5.0,5.0,25.0,98.0,166.0,133.0,...,0.0,0.0,10.0,18.0,18.0,39.0,54.0,13.0,29.0,22.0
2014-12-31,119.0,1760.450,28.0,28.0,11.0,17.0,56.0,243.0,455.0,344.0,...,0.0,0.0,25.0,82.0,47.0,145.0,152.0,46.0,67.0,67.0
2015-12-31,200.0,3054.270,49.0,53.0,23.0,30.0,102.0,445.0,803.0,612.0,...,0.0,0.0,49.0,123.0,82.0,235.0,261.0,106.0,323.0,268.0
2016-12-31,212.0,3331.087,44.0,55.0,25.0,30.0,99.0,447.0,820.0,618.0,...,0.0,0.0,45.0,123.0,94.0,272.0,267.0,135.0,539.0,475.0
2017-12-31,209.0,3567.807,53.0,61.0,33.0,28.0,114.0,477.0,863.0,671.0,...,1.0,1.0,49.0,104.0,125.0,250.0,287.0,173.0,1088.0,999.0


In [104]:
allmeanstats['Tomas Hertl']

,GP,TOI,Goals,Total Assists,First Assists,Second Assists,Total Points,Shots,iCF,iFF,...,Penalties Drawn,Giveaways,Takeaways,Hits,Hits Taken,Shots Blocked,Faceoffs Won,Faceoffs Lost,SH%,Faceoffs %
Season_index,,,,,,,,,,,,,,,,,,,,,
2013-12-31,37.000000,567.150000,15.000000,10.000000,5.000000,5.000000,25.0,98.000000,166.000000,133.000000,...,10.000000,18.000000,18.000000,39.000000,54.000000,13.000000,29.000000,22.000000,6.533333,0.568627
2014-12-31,59.500000,880.225000,14.000000,14.000000,5.500000,8.500000,28.0,121.500000,227.500000,172.000000,...,12.500000,41.000000,23.500000,72.500000,76.000000,23.000000,33.500000,33.500000,8.678571,0.500000
2015-12-31,66.666667,1018.090000,16.333333,17.666667,7.666667,10.000000,34.0,148.333333,267.666667,204.000000,...,16.333333,41.000000,27.333333,78.333333,87.000000,35.333333,107.666667,89.333333,9.081633,0.546531
2016-12-31,70.666667,1110.362333,14.666667,18.333333,8.333333,10.000000,33.0,149.000000,273.333333,206.000000,...,15.000000,41.000000,31.333333,90.666667,89.000000,45.000000,179.666667,158.333333,10.159091,0.531558
2017-12-31,69.666667,1189.269000,17.666667,20.333333,11.000000,9.333333,38.0,159.000000,287.666667,223.666667,...,16.333333,34.666667,41.666667,83.333333,95.666667,57.666667,362.666667,333.000000,9.000000,0.521322


In [39]:
th = allstats['Tomas Hertl'].copy()
thsum = allsumstats['Tomas Hertl'].copy()
thmean = allmeanstats['Tomas Hertl'].copy()

In [48]:
scols = []
for i in thsum.columns:
    scols.append('sum ' + i)
thsum.columns = scols

In [49]:
thsum.columns

Index(['sum GP', 'sum TOI', 'sum Goals', 'sum Total Assists',
       'sum First Assists', 'sum Second Assists', 'sum Total Points',
       'sum Shots', 'sum iCF', 'sum iFF', 'sum iSCF', 'sum iHDCF',
       'sum Rush Attempts', 'sum Rebounds Created', 'sum PIM',
       'sum Total Penalties', 'sum Minor', 'sum Major', 'sum Misconduct',
       'sum Penalties Drawn', 'sum Giveaways', 'sum Takeaways', 'sum Hits',
       'sum Hits Taken', 'sum Shots Blocked', 'sum Faceoffs Won',
       'sum Faceoffs Lost'],
      dtype='object')

In [105]:
scols, mcols = [], []
#Relabeling sum and mean columns for clarity, using Hertl for convenience, could be any player
for i in allsumstats['Tomas Hertl'].columns:
    scols.append('sum '+i)
for i in allmeanstats['Tomas Hertl'].columns:
    mcols.append('mean '+i)

for p in allstats:
    allsumstats[p].columns = scols
    allmeanstats[p].columns = mcols
    allstats[p] = pd.concat([allstats[p], allsumstats[p], allmeanstats[p]], axis=1)

In [106]:
allstats['Tomas Hertl']

,Season_Player,Player,Season,Position,GP,TOI,Goals,Total Assists,First Assists,Second Assists,...,mean Penalties Drawn,mean Giveaways,mean Takeaways,mean Hits,mean Hits Taken,mean Shots Blocked,mean Faceoffs Won,mean Faceoffs Lost,mean SH%,mean Faceoffs %
Season_index,,,,,,,,,,,,,,,,,,,,,
2013-12-31,2013 Tomas Hertl,Tomas Hertl,2013,C,37,567.150,15,10,5,5,...,10.000000,18.000000,18.000000,39.000000,54.000000,13.000000,29.000000,22.000000,6.533333,0.568627
2014-12-31,2014 Tomas Hertl,Tomas Hertl,2014,C,82,1193.300,13,18,6,12,...,12.500000,41.000000,23.500000,72.500000,76.000000,23.000000,33.500000,33.500000,8.678571,0.500000
2015-12-31,2015 Tomas Hertl,Tomas Hertl,2015,C,81,1293.820,21,25,12,13,...,16.333333,41.000000,27.333333,78.333333,87.000000,35.333333,107.666667,89.333333,9.081633,0.546531
2016-12-31,2016 Tomas Hertl,Tomas Hertl,2016,C,49,843.967,10,12,7,5,...,15.000000,41.000000,31.333333,90.666667,89.000000,45.000000,179.666667,158.333333,10.159091,0.531558
2017-12-31,2017 Tomas Hertl,Tomas Hertl,2017,C,79,1430.020,22,24,14,10,...,16.333333,34.666667,41.666667,83.333333,95.666667,57.666667,362.666667,333.000000,9.000000,0.521322


In [107]:
players = list(allstats.keys())


In [108]:
allallstats = pd.DataFrame()

In [109]:
allallstats

""


In [110]:
for p in players:
    allallstats = pd.concat([allallstats, allstats[p].set_index(allstats[p].Season_Player)])

In [111]:
allallstats[allallstats.Season == 2009].sample(10)

,Season_Player,Player,Season,Position,GP,TOI,Goals,Total Assists,First Assists,Second Assists,...,mean Penalties Drawn,mean Giveaways,mean Takeaways,mean Hits,mean Hits Taken,mean Shots Blocked,mean Faceoffs Won,mean Faceoffs Lost,mean SH%,mean Faceoffs %
Season_Player,,,,,,,,,,,,,,,,,,,,,
2009 James Neal,2009 James Neal,James Neal,2009,"L, R",78,1419.2500,27,28,18,10,...,35.000000,26.000000,62.000000,184.000000,90.500000,29.500000,15.000000,30.500000,7.274510,0.329670
2009 Andrej Sekera,2009 Andrej Sekera,Andrej Sekera,2009,D,49,855.0330,4,7,3,4,...,7.666667,29.000000,15.000000,42.000000,88.666667,63.666667,0.000000,0.666667,19.000000,0.000000
2009 Jason Spezza,2009 Jason Spezza,Jason Spezza,2009,C,60,1143.7700,23,34,22,12,...,23.666667,78.333333,52.000000,22.333333,66.333333,17.000000,676.666667,636.666667,6.977528,0.515228
2009 Matt Martin,2009 Matt Martin,Matt Martin,2009,L,5,66.1833,0,2,2,0,...,4.000000,3.000000,1.000000,15.000000,9.000000,3.000000,0.000000,0.000000,inf,NaN
2009 Alexander Semin,2009 Alexander Semin,Alexander Semin,2009,R,73,1396.0000,40,44,26,18,...,31.666667,68.333333,58.000000,24.000000,73.333333,15.000000,7.333333,9.666667,6.860000,0.431373
2009 Radim Vrbata,2009 Radim Vrbata,Radim Vrbata,2009,R,82,1330.2700,24,19,13,6,...,10.000000,16.000000,22.333333,11.000000,40.000000,15.333333,3.333333,6.666667,10.240741,0.333333
2009 Taylor Chorney,2009 Taylor Chorney,Taylor Chorney,2009,D,42,730.4500,0,3,2,1,...,1.500000,16.500000,5.000000,18.000000,32.000000,39.000000,0.000000,0.000000,inf,NaN
2009 Michael Frolik,2009 Michael Frolik,Michael Frolik,2009,R,82,1433.2300,21,22,7,15,...,19.000000,32.500000,38.500000,59.000000,136.000000,28.000000,20.000000,31.000000,8.976190,0.392157
2009 Duncan Keith,2009 Duncan Keith,Duncan Keith,2009,D,82,2181.3200,14,55,23,32,...,11.000000,46.666667,49.000000,51.000000,80.333333,116.666667,0.000000,0.000000,15.705882,NaN


In [38]:
contracts[contracts.Player == 'Tomas Hertl']

,player_id,first_name,last_name,position,birthdate,season,team_id,team_name,cap_hit,active,...,contract_type,contract_end,ufa_year,total_value,Player,signing_year,signing_age,signing_year_cap,cap_pct,signing_status
3447,4791,Tomas,Hertl,Center,1993-11-12,2016,21.0,Sharks,3000000,1.0,...,history,2017,2021,6000000,Tomas Hertl,2016,22.0,73000000,4.11,RFA
3449,4791,Tomas,Hertl,Center,1993-11-12,2018,21.0,Sharks,5625000,1.0,...,current,2021,2021,22500000,Tomas Hertl,2018,24.0,79500000,7.08,RFA


In [77]:
contracts.signing_year.value_counts()

2017    413
2018    411
2016    350
2015    271
2014    132
2013     75
2012     40
2019     28
2011     21
2010     12
2009      6
2008      3
Name: signing_year, dtype: int64

In [113]:
contracts[contracts.signing_year < 2009]

,player_id,first_name,last_name,position,birthdate,season,team_id,team_name,cap_hit,active,...,contract_type,contract_end,ufa_year,total_value,Player,signing_year,signing_age,signing_year_cap,cap_pct,signing_status
155,814,Eric,Staal,Center,1984-10-29,2015,29.0,Wild,8250000,1.0,...,history,2015,2011,57750000,Eric Staal,2008,23.0,56700000,14.55,RFA
261,1216,Alex,Ovechkin,Left Wing,1985-09-17,2015,15.0,Capitals,9538462,1.0,...,current,2020,2012,124000000,Alex Ovechkin,2008,22.0,56700000,16.82,RFA
683,2039,Anze,Kopitar,Center,1987-08-24,2015,23.0,Kings,6800000,1.0,...,history,2015,2013,47600000,Anze Kopitar,2008,21.0,56700000,11.99,RFA


In [114]:
table = contracts[contracts.signing_year > 2009].set_index('contract_id')

In [115]:
table['year_match'] = table.signing_year - 1

1089    2018
192     2014
348     2014
1667    2017
1344    2016
879     2014
1700    2016
1018    2015
1005    2013
1535    2014
Name: year_match, dtype: int64

In [116]:
table = pd.merge(table, allallstats, 
                how = 'left', left_on = ['Player', 'year_match'], 
                right_on = ['Player', 'Season'])

In [117]:
table.sample(10)

,player_id,first_name,last_name,position,birthdate,season,team_id,team_name,cap_hit,active,...,mean Penalties Drawn,mean Giveaways,mean Takeaways,mean Hits,mean Hits Taken,mean Shots Blocked,mean Faceoffs Won,mean Faceoffs Lost,mean SH%,mean Faceoffs %
672,3767,Garrett,Wilson,Left Wing,1991-03-16,2017,7.0,Penguins,650000,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1577,6019,TJ,Tynan,Right Wing,1992-02-25,2016,269.0,Golden Knights,600000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
950,4234,Korbinian,Holzer,Defense,1988-02-16,2015,25.0,Ducks,750000,1.0,...,3.333333,14.333333,5.000000,59.666667,41.666667,36.666667,0.000000,0.000000,24.500000,NaN
972,4320,Mark,Borowiecki,Defense,1989-07-12,2015,9.0,Senators,1100000,1.0,...,5.333333,3.333333,0.000000,23.666667,16.000000,6.000000,0.000000,0.000000,8.000000,NaN
1109,4448,Andy,Andreoff,Center,1991-05-17,2017,17.0,Lightning,677500,NaN,...,10.000000,8.666667,8.000000,61.666667,38.666667,6.000000,69.000000,63.666667,9.800000,0.520101
1189,4608,Mike,Sislo,Right Wing,1988-01-20,2015,4.0,Islanders,562500,0.0,...,1.000000,1.000000,3.000000,19.000000,7.000000,4.000000,5.000000,1.000000,inf,0.833333
287,2535,James,Neal,Left Wing,1987-09-03,2018,18.0,Flames,5750000,1.0,...,24.666667,29.000000,46.000000,80.333333,68.666667,26.333333,3.666667,8.000000,8.506329,0.314286
1124,4464,Jean-Gabriel,Pageau,Center,1992-11-11,2017,9.0,Senators,3100000,1.0,...,17.666667,25.666667,24.666667,161.666667,110.666667,53.000000,502.333333,456.333333,9.731707,0.523992
1311,4810,Nicolas,Kerdiles,Left Wing,1994-01-11,2018,28.0,Jets,650000,0.0,...,0.000000,1.000000,0.000000,0.500000,1.000000,1.000000,0.500000,0.500000,NaN,0.500000
745,3893,Jarred,Tinordi,Defense,1992-02-20,2016,27.0,Predators,700000,0.0,...,5.000000,5.666667,0.666667,28.000000,25.666667,18.666667,0.000000,0.000000,inf,NaN


In [119]:
list(table.columns)

['player_id',
 'first_name',
 'last_name',
 'position',
 'birthdate',
 'season',
 'team_id',
 'team_name',
 'cap_hit',
 'active',
 'signing_date',
 'signing_team',
 'length',
 'contract_level',
 'expiry_status',
 'contract_type',
 'contract_end',
 'ufa_year',
 'total_value',
 'Player',
 'signing_year',
 'signing_age',
 'signing_year_cap',
 'cap_pct',
 'signing_status',
 'year_match',
 'Season_Player',
 'Season',
 'Position',
 'GP',
 'TOI',
 'Goals',
 'Total Assists',
 'First Assists',
 'Second Assists',
 'Total Points',
 'Shots',
 'iCF',
 'iFF',
 'iSCF',
 'iHDCF',
 'Rush Attempts',
 'Rebounds Created',
 'PIM',
 'Total Penalties',
 'Minor',
 'Major',
 'Misconduct',
 'Penalties Drawn',
 'Giveaways',
 'Takeaways',
 'Hits',
 'Hits Taken',
 'Shots Blocked',
 'Faceoffs Won',
 'Faceoffs Lost',
 'Season_index',
 'sum GP',
 'sum TOI',
 'sum Goals',
 'sum Total Assists',
 'sum First Assists',
 'sum Second Assists',
 'sum Total Points',
 'sum Shots',
 'sum iCF',
 'sum iFF',
 'sum iSCF',
 'sum iHD

In [123]:
table_c = table[['Player', 'Season', 'position', 'signing_age', 'signing_date', 
                'Total Points', 'mean Total Points', 'sum TOI', 
                'cap_hit', 'cap_pct', 'length']]

In [128]:
contracts[contracts['Player'] == 'Brooks Laich']

,player_id,first_name,last_name,position,birthdate,season,team_id,team_name,cap_hit,active,...,contract_type,contract_end,ufa_year,total_value,Player,signing_year,signing_age,signing_year_cap,cap_pct,signing_status
30,220,Brooks,Laich,Center,1983-06-23,2015,NaN,NaN,4500000,0.0,...,history,2016,2010,27000000,Brooks Laich,2011,28.0,64300000,7.00,UFA
32,220,Brooks,Laich,Center,1983-06-23,2017,NaN,NaN,650000,0.0,...,history,2017,2010,650000,Brooks Laich,2017,34.0,75000000,0.87,UFA


In [127]:
allallstats[allallstats.Player == 'Brooks Laich']

,Season_Player,Player,Season,Position,GP,TOI,Goals,Total Assists,First Assists,Second Assists,...,mean Penalties Drawn,mean Giveaways,mean Takeaways,mean Hits,mean Hits Taken,mean Shots Blocked,mean Faceoffs Won,mean Faceoffs Lost,mean SH%,mean Faceoffs %
Season_Player,,,,,,,,,,,,,,,,,,,,,
2007 Brooks Laich,2007 Brooks Laich,Brooks Laich,2007,C,82,1151.750,21,16,13,3,...,26.000000,26.000000,22.000000,83.000000,119.000000,56.000000,281.000000,315.000000,5.809524,0.471477
2008 Brooks Laich,2008 Brooks Laich,Brooks Laich,2008,C,82,1417.250,23,30,18,12,...,29.500000,37.500000,33.000000,91.000000,134.500000,49.000000,271.000000,282.500000,6.977273,0.489612
2009 Brooks Laich,2009 Brooks Laich,Brooks Laich,2009,C,78,1426.470,25,34,19,15,...,28.000000,37.666667,36.666667,91.666667,138.666667,45.666667,231.333333,250.000000,7.666667,0.480609
2010 Brooks Laich,2010 Brooks Laich,Brooks Laich,2010,C,82,1504.250,16,32,19,13,...,25.666667,39.666667,42.000000,101.666667,142.333333,45.000000,227.333333,230.000000,9.593750,0.497085
2011 Brooks Laich,2011 Brooks Laich,Brooks Laich,2011,C,82,1516.520,16,25,20,5,...,21.333333,33.666667,44.666667,108.666667,130.666667,61.666667,361.666667,390.000000,10.877193,0.481153
2012 Brooks Laich,2012 Brooks Laich,Brooks Laich,2012,C,9,148.833,1,3,2,1,...,13.333333,22.666667,30.666667,82.333333,86.000000,50.666667,324.666667,341.666667,12.363636,0.487244
2013 Brooks Laich,2013 Brooks Laich,Brooks Laich,2013,C,51,879.833,8,7,5,2,...,8.666667,17.000000,24.666667,65.333333,74.666667,44.333333,302.666667,342.333333,11.040000,0.469251
2014 Brooks Laich,2014 Brooks Laich,Brooks Laich,2014,C,66,971.733,7,13,7,6,...,5.666667,15.000000,15.666667,61.000000,73.666667,31.333333,110.333333,136.000000,11.937500,0.447903
2015 Brooks Laich,2015 Brooks Laich,Brooks Laich,2015,C,81,926.233,2,12,6,6,...,9.333333,18.666667,24.666667,98.333333,95.666667,40.666667,129.333333,171.000000,16.588235,0.430633


In [125]:
table_c.head()

,Player,Season,position,signing_age,signing_date,Total Points,mean Total Points,sum TOI,cap_hit,cap_pct,length
0,Brooks Laich,2010.0,Center,28.0,2011-06-28,48.0,53.333333,4347.970,4500000,7.00,6
1,Brooks Laich,NaN,Center,34.0,2017-10-19,NaN,NaN,NaN,650000,0.87,1
2,Francois Beauchemin,2014.0,Defense,35.0,2015-07-01,23.0,21.333333,4198.230,4500000,6.30,3
3,Francois Beauchemin,2016.0,Defense,37.0,2017-08-21,18.0,25.000000,5254.620,1000000,1.33,1
4,Patrice Bergeron,2012.0,Center,27.0,2013-07-12,32.0,51.000000,3746.567,6875000,10.69,8


In [129]:
table_c.Season.isnull().sum()

309

In [130]:
table_c.shape

(1753, 11)

In [131]:
table_c.dropna(inplace=True)

/Users/colinshanahan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [132]:
table_c.shape

(1444, 11)

In [134]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

In [141]:
table_c['skater'] = table_c.position != 'Goaltender'
table_c['forward'] = (table_c.position != 'Goaltender') & (table_c.position != 'Defense')
table_c.replace([True, False], [1, 0], inplace=True)



/Users/colinshanahan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/colinshanahan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/colinshanahan/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3795: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.

In [142]:
y = table_c.cap_pct
X = table_c[['forward', 'signing_age', 'Total Points', 'mean Total Points', 'sum TOI']]

In [143]:
X = sm.add_constant(X)

In [144]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 20)

In [145]:
model = sm.OLS(y_train.values, X_train.values)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.788
Model:                            OLS   Adj. R-squared:                  0.787
Method:                 Least Squares   F-statistic:                     855.1
Date:                Thu, 28 Mar 2019   Prob (F-statistic):               0.00
Time:                        09:26:55   Log-Likelihood:                -2031.6
No. Observations:                1155   AIC:                             4075.
Df Residuals:                    1149   BIC:                             4105.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.8715      0.325     11.917      0.000       3.234       4.509
x1            -1.0174      0.104     -9.804      0.000      -1.221      -0.814
x2            -0.1250      0.013     -9.635      0.000      -0.150      -0.100
x3             0.0244      0.005      4.582      0.000       0.014       0.035
x4             0.1152      0.007     15.845      0.000       0.101       0.129
x5             0.0003    5.4e-05      6.332      0.000       0.000       0.000
==============================================================================
Omnibus:                       80.067   Durbin-Watson:                   2.028
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              154.364
Skew:                           0.466   Prob(JB):                     3.02e-34
Kurtosis:                       4.530   Cond. No.                     2.03e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.03e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [146]:
def rmse(yhat, y):
    return np.sqrt(((yhat - y)**2).mean())

In [147]:
preds = results.predict(X_test)

In [148]:
rmse(preds, y_test)

1.376953328072369